# Pandas: CSV failų nuskaitymas iš kompiuterio ir iš URL

Šiame faile pateikiami praktiniai pavyzdžiai, kaip su `pandas` nuskaityti CSV duomenis:

- iš kompiuteryje esančio failo (Windows / macOS / Linux)
- iš interneto nuorodos (URL)

Pateikiamos dažniausios klaidos, jų priežastys ir gerosios praktikos.


In [3]:
import os
from pathlib import Path
import pandas as pd

pd.__version__


'3.0.0'

## 1. Darbo katalogas ir failų paieška

Dažna problema – failas yra „teisingoje vietoje“, bet Python jo neranda, nes paleidimo katalogas skiriasi.

- `os.getcwd()` parodo, kurioje vietoje šiuo metu ieškoma failų, kai nurodomas santykinis kelias.
- `Path(...).exists()` leidžia patikrinti, ar kelias teisingas.


In [4]:
os.getcwd()


'c:\\Users\\donat\\iCloudDrive\\Documents\\BIT Data Analytics\\V Python\\2026-02-03\\python tutorials\\pandas'

In [8]:
pwd

'c:\\Users\\donat\\iCloudDrive\\Documents\\BIT Data Analytics\\V Python\\2026-02-03\\python tutorials\\pandas'

C:\Users\User\OneDrive\Desktop\BIT\2025 11 12 DUOM\V Modulis_Python_2025 11 12 DUOM\BIT_PYTHON

In [11]:
# Pavyzdys: patikrinimas, ar failas egzistuoja (pakeisti kelią į savo)
example_path = Path("C:\\Users\\donat\\iCloudDrive\\Documents\\BIT Data Analytics\\V Python\\Files\\dirty_employees.csv")

#'c:\\Users\\User\\OneDrive\\Desktop\\BIT\\2025 11 12 DUOM\\V Modulis_Python_2025 11 12 DUOM\\BIT_PYTHON\\Python_P\\Pandas Course Resources\\retail\\oil.csv'
example_path.exists()


True

## 2. CSV nuskaitymas iš kompiuterio

### 2.1. Pagrindinis variantas

Dažniausiai pakanka `pd.read_csv(path)`.

### Dažnos klaidos (Windows):

- **SyntaxError dėl kelio**: `\` yra escape ženklas. Patikimiausia naudoti `/` arba `r"..."`.
- **FileNotFoundError**: neteisingas kelias, failo pavadinimas, plėtinys arba darbo katalogas.
- **PermissionError**: failas atidarytas Excel programoje arba nėra teisių pasiekti aplanką.


In [ ]:
# Rekomenduojamas kelio formatas Windows aplinkoje – su /
file_path = "C:\\Users\\donat\\iCloudDrive\\Documents\\BIT Data Analytics\\V Python\\Files\\dirty_employees.csv"

# Jei kelias neteisingas, bus FileNotFoundError
# df = pd.read_csv(file_path)

# Patogu pirma pasitikrinti:
Path(file_path).exists()


True

In [9]:
# Alternatyva: raw string (tinka, kai norisi naudoti \)
file_path_raw = r"C:\Users\donat\iCloudDrive\Documents\BIT Data Analytics\V Python\Files\dirty_employees.csv"
Path(file_path_raw).exists()


True

### 2.2. Geroji praktika: `pathlib` ir aiškus kelio sudarymas

`pathlib.Path` padeda tvarkingai konstruoti kelius ir sumažina klaidų tikimybę.


In [15]:
base_dir = Path("C:\\Users\\donat\\iCloudDrive\\Documents\\BIT Data Analytics\\V Python\\Files")
csv_file = base_dir / "dirty_employees.csv"

csv_file, csv_file.exists()


(WindowsPath('C:/Users/donat/iCloudDrive/Documents/BIT Data Analytics/V Python/Files/dirty_employees.csv'),
 True)

In [16]:
# Nuskaitymas (paleisti tik tada, kai kelias tikrai egzistuoja)
df = pd.read_csv(csv_file)
df.head()


,client_id,full_name,email,birthdate,phone,status,country
0,1,Brandy odonnell,ishepherd@rivera-lawson.info,1963-04-09,874-743-5327x728,INACTIVE,NaN
1,2,Jessica nolan,lynchrebecca @yahoo.com,not_a_date,NaN,NaN,NaN
2,3,KAITLYN GARCIA,melissa92 @fisher.info,NaN,NaN,Active,NaN
3,4,Johnny Dillon,vunderwood@holt.com,NaN,695-157-0122x44232,active,NaN
4,5,William cantu,davisdanielfritz-robbins.com,03/10/1966,1237820728,NaN,Solomon Islands


## 3. Dažniausi `read_csv` parametrai

CSV failai realiame pasaulyje dažnai turi nestandartinį formatą. Šie parametrai padeda išspręsti tipines problemas.

- `sep` – skyriklis (dažnai `,` arba `;`)
- `encoding` – koduotė (pvz. `utf-8`, `utf-8-sig`, `cp1257`, `latin1`)
- `decimal` – dešimtainis skirtukas (pvz. `.` arba `,`)
- `parse_dates` – datos konvertavimas į `datetime`
- `dtype` – stulpelių tipų nurodymas, kai automatinis atpažinimas neteisingas
- `na_values` – reikšmės, kurios laikomos trūkstamomis (`NA`, `N/A`, `null`, ir pan.)
- `usecols` – nuskaityti tik dalį stulpelių (greičiau ir mažiau klaidų)


In [2]:
# Pavyzdys: skyriklis ';', dešimtainis ',' ir datų konvertavimas
# df = pd.read_csv(
#     csv_file,
#     sep=';',
#     decimal=',',
#     encoding='utf-8',
#     parse_dates=['date']
# )
# df.info()


## 4. Tipinės klaidos ir kaip jas atpažinti

### 4.1. `FileNotFoundError`
Priežastys:
- neteisingas kelias
- neteisingas failo pavadinimas / plėtinys
- maišomas darbo katalogas

Sprendimas:
- patikrinti `os.getcwd()`
- patikrinti `Path(path).exists()`

### 4.2. `ParserError`
Priežastys:
- neteisingas `sep`
- sugadintas CSV (nelygus stulpelių skaičius eilutėse)

Sprendimai:
- pabandyti `sep=';'` ar kitą skyriklį
- naudoti `on_bad_lines='skip'` (kai galima praleisti blogas eilutes)

### 4.3. `UnicodeDecodeError`
Priežastis:
- neteisinga `encoding`

Sprendimas:
- pabandyti `encoding='utf-8-sig'`, `latin1`, `cp1257`


In [ ]:
# Pavyzdys: saugus nuskaitymas su klaidų gaudymu
path = "C:/Users/User/OneDrive/Dokumentai/data/transactions.csv"

try:
    df = pd.read_csv(path)
    display(df.head())
except FileNotFoundError as e:
    print("Failas nerastas. Patikrinti kelią ir darbo katalogą.")
    print(e)
except PermissionError as e:
    print("Nėra prieigos prie failo. Patikrinti, ar failas neatidarytas kitoje programoje.")
    print(e)
except UnicodeDecodeError as e:
    print("Koduotės klaida. Pabandyti encoding='utf-8-sig' arba 'latin1'.")
    print(e)


## 5. CSV nuskaitymas iš URL

`pd.read_csv()` gali nuskaityti CSV tiesiai iš interneto, jei URL grąžina „raw“ CSV turinį.

Dažnos klaidos:
- URL nukreipia į HTML puslapį, o ne į CSV (gaunamas vienas tekstinis stulpelis arba `ParserError`)
- reikalinga autentifikacija (pvz., privatūs failai)

Geroji praktika:
- naudoti „raw“ nuorodą (pvz., GitHub Raw)
- patikrinti, ar URL atsidaro naršyklėje kaip CSV tekstas


In [8]:
# Įrašyti realų URL, kuris grąžina CSV turinį
url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv"

df_url = pd.read_csv(url)
df_url.head()



,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF


### 5.1. GitHub: „raw“ nuoroda

GitHub puslapio nuoroda dažnai grąžina HTML. Reikia „raw“ nuorodos.

Pavyzdys (logika):
- vietoje `github.com/.../blob/.../file.csv`
- naudoti `raw.githubusercontent.com/.../.../file.csv`

Jei nuskaitymas grąžina vieną stulpelį su `<html>...`, vadinasi naudojama netinkama nuoroda.


In [11]:
# Pavyzdinė struktūra (pakeisti į realų failą)
# github_raw_url = "https://raw.githubusercontent.com/USER/REPO/BRANCH/path/to/file.csv"

# df_gh = pd.read_csv(github_raw_url)
# df_gh.head()


### 5.2. Patikrinimas, ar URL grąžina CSV, o ne HTML

Kai kyla įtarimas, kad URL grąžina HTML, verta peržiūrėti pirmus kelis šimtus simbolių.
Jei matomas `<html>` arba kitas puslapio turinys, tai nėra tinkamas CSV šaltinis.


In [ ]:
import urllib.request

test_url = "https://example.com/data.csv"

try:
    with urllib.request.urlopen(test_url) as resp:
        preview = resp.read(200).decode("utf-8", errors="replace")
    print(preview)
except Exception as e:
    print("Nepavyko nuskaityti URL turinio. Priežastis gali būti ryšys arba prieigos ribojimai.")
    print(e)


## 6. Dideli CSV failai: našumas ir atmintis

Kai failas didelis, verta:

- nuskaityti tik reikalingus stulpelius (`usecols`)
- nuskaityti dalimis (`chunksize`)
- nurodyti `dtype`, kad sumažėtų automatinio tipų spėjimo klaidos


In [ ]:
# Pavyzdys su chunksize (paleisti turint realų kelią)
# path = "C:/Users/User/OneDrive/Dokumentai/data/big_file.csv"
# chunks = pd.read_csv(path, chunksize=100_000)

# total_rows = 0
# for chunk in chunks:
#     total_rows += len(chunk)
# total_rows


## 7. Greitas patikrinimas po nuskaitymo (rekomenduojama seka)

Po sėkmingo `read_csv()` verta visada atlikti:

- `df.shape` – ar eilučių/stulpelių skaičius tikėtinas
- `df.head()` – ar turinys atrodo teisingas
- `df.info()` – ar tipai teisingi, ar nėra trūkstamų reikšmių
- `df.isna().sum()` – trūkstamų reikšmių santrauka (jei aktualu)


In [ ]:
# Pavyzdinis šablonas (paleisti, kai df jau nuskaitytas)
# df.shape
# df.head()
# df.info()
# df.isna().sum()
